In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import folium

## Load data  into a Pandas dataframe

In [ ]:
complete_dataset = pd.read_csv('../data/fao_data_crops_data.csv')




In [ ]:
# We split the data and metadata and store them in 'crops' and 'flags' dataframe, respecitvely.
crops = complete_dataset.loc[:2255342].copy() 
flags = complete_dataset.loc[2255344:2255348].copy() 
# 'flags' contains correspondance list of acronyms that describe how a given sample was acquired --> only informative
flags.drop(['element','year','unit','value','value_footnotes','category'], axis=1, inplace = True) 
flags.rename(columns={'country_or_area':'acronym', 'element_code':'description'}, inplace=True) 
flags.set_index('acronym', inplace=True)
flags

## Exploratory data analysis

In [ ]:
crops.head()

In [ ]:
print("Size of the DataFrame: {s}\n".format(s=crops.shape))
print("Variable types present in DataFrame: \n{t}".format(t=crops.dtypes))

In [ ]:
# List all the different footnotes values present in the dataset
footnotes = crops['value_footnotes'].unique() 
print(footnotes)
# Display dataframe that only contains one given value of 'value_footnotes'
display(crops.query('value_footnotes==@footnotes[4]')) 
# Return dataframe that only contains samples having NaN as value for 'value_footnotes'
crops[crops.value_footnotes.isnull()] 

In [ ]:
print(crops['element'].unique())
print(crops['year'].unique())
print(crops['unit'].unique())
print(crops['category'].unique())
print(crops['element_code'].unique())
print(crops['country_or_area'].unique())

## Data preprocessing

In [ ]:
# Returns a boolean of whether a column contains NaN (True) or not (False).
print(crops.isnull().values.any(axis=0)) 

# Drop rows which contain only missing values.
crops.dropna(how='all', inplace=True) 



In [ ]:
# We drop the samples where 'value' is unknown (NaN) because they are of no utility    
crops.dropna(subset=['value', 'value_footnotes'], inplace=True) 

# Let's drop also all the samples that have 'NR' as a 'value_footnotes' value or 0 as 'value'
crops.drop(index=crops[crops['value_footnotes'].str.contains('NR')].index, inplace=True)
crops.drop(index=crops[crops['value']==0].index, inplace=True)


Separate regions from countries

In [ ]:
regions_bool = crops['country_or_area'].str.contains('\+')
crops_regions = crops[regions_bool].copy()
crops_countries = crops[~regions_bool].copy()
crops_countries[crops_countries.country_or_area.str.contains('China')].tail(60)

In [ ]:
crops_countries_by_country_year = crops_countries.groupby(['country_or_area', 'element']) \
                            .agg({'value':'mean'}) \
                            .rename(columns={'value':'mean_'}) 
                            #.sort_values(by='value',ascending=False)
crops_countries_by_country_year

In [ ]:
area_harvested = crops_countries_by_country_year.loc[(slice(None),'Area Harvested'), :]
#area_harvested.mean_.argmax()
area_harvested.loc['United States of America']

In [ ]:
surface_country = pd.read_csv('../data/API_AG.LND.TOTL.K2_DS2_en_csv_v2_422954.csv', skiprows=3)
surface_country.set_index('Country Name', inplace = True)
crops_countries_area = area_harvested.join(surface_country['2018'], on='country_or_area', how='left')
crops_countries_area['ratio'] = area_harvested['mean_']/(crops_countries_area['2018']*100)
crops_countries_area.dropna(inplace=True)
#plt.bar(crops_countries_area_mean_nobermuda.index.get_level_values(level='country_or_area').values, height='ratio')
#plt.show()
pl = crops_countries_area.plot(kind="bar", 
                             y="ratio", 
                            figsize=(10, 7), alpha=0.5, color="olive")

In [ ]:
crops_countries_area

In [ ]:
m = folium.Map()
world_geo = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'
folium.Choropleth(
    geo_data=world_geo,
    name='choropleth',
    data=crops_countries_area.ratio,
    columns=[crops_countries_area.index.get_level_values(level='country_or_area').values, 'ratio'],
    key_on='feature.properties.name',
    bins = [np.histogram_bin_edges(crops_countries_area.ratio, bins=10)],
    #fill_color='GnBu',
    #fill_opacity=0.9,
    #line_opacity=0.2,
    #highlight = True,
).add_to(m)
#https://github.com/johan/world.geo.json/blob/master/countries.geo.json
m